<a href="https://colab.research.google.com/github/Davjot/Conversational-AI-Assignments/blob/main/Decision_Tree_ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/weather.csv')

In [2]:
def find_entropy(df):
  target = df.keys()[-1]
  target_values = df[target].unique()
  entropy = 0
  for value in target_values:
   prob = len(df[df[target] == value])/len(df)
   entropy += -(prob * np.log2(prob))
  return entropy 

In [4]:
find_entropy(df)

0.9402859586706309

In [17]:
def find_avg_info_entropy(df, attribute):
  attr_values = df[attribute].unique()
  target = df.keys()[-1]
  target_values = df[target].unique()
  avg_info_entropy = 0
  for value1 in attr_values:
    entropy_subsample = 0
    for value2 in target_values:
      num = len(df[attribute][df[attribute] == value1][df[target] == value2])
      den = len(df[attribute][df[attribute] == value1])
      prob = num/den
      entropy_subsample += -(prob * np.log2(prob + 1e-7))
    weight = den/len(df)
    avg_info_entropy += weight * entropy_subsample
  return avg_info_entropy

In [18]:
find_avg_info_entropy(df, 'Outlook')

0.6935358915770655

In [15]:
def find_winner(df):
  attributes = df.keys()[:-1]
  IG = []
  for attribute in attributes:
    IG.append(find_entropy(df) - find_avg_info_entropy(df, attribute))
  return df.keys()[:-1][np.argmax(IG)]

In [19]:
find_winner(df)

'Outlook'

In [22]:
def training(df, tree = None):
  target = df.keys()[-1]
  attribute = find_winner(df)
  attr_values = df[attribute].unique()
  if tree is None:
    tree = {}
    tree[attribute] = {}
  for value in attr_values:
    sub_df = df[df[attribute] == value].reset_index(drop = True)
    clvalue, count = np.unique(sub_df[target], return_counts = True)
    if len(count) == 1:
      tree[attribute][value] = clvalue[0]
    else:
      tree[attribute][value] = training(sub_df)
  return tree

In [23]:
tree = training(df)

In [24]:
tree

{'Outlook': {'rainy': {'Humidity': {'high': 0, 'normal': 1}},
  'overcast': 1,
  'sunny': {'Windy': {0: 1, 1: 0}}}}

In [25]:
def prediction(instance, tree):
  for attribute in tree.keys():
    value = instance[attribute]
    tree = tree[attribute][value]
    if type(tree) is dict:
      predict = prediction(instance, tree)
    else:
      predict = tree
      break
  return predict

In [26]:
df1 = pd.read_csv('/content/weather_test.csv')

In [27]:
y_predict = []
for i in range(len(df1)):
  instance = df1.iloc[i,:]
  predict = prediction(instance, tree)
  y_predict.append(predict)

In [28]:
y_predict

[0, 1]